In [1]:
from IPython.display import HTML, display
import tabulate
table = [["Sun",696000,1989100000],
         ["Earth",6371,5973.6],
         ["Moon",1737,73.5],
         ["Mars",3390,641.85]]
display(HTML(tabulate.tabulate(table, tablefmt='html')))


Sun,696000,1.9891e+09
Earth,6371,5973.6
Moon,1737,73.5
Mars,3390,641.85


In [5]:
#Defines basic, helpful types such as vectors

import math

class PolarVector:
    def __init__(self, magnitude, theta):
        self.magnitude = magnitude
        self.theta = theta
        
    def __mul__(self, other):
        return PolarVector(self.magnitude * other, self.theta)
    
    def __truediv__(self, other):
        return PolarVector(self.magnitude / other, self.theta)
    
    def __add__(self, other):
        return PolarVector(self.magnitude + other.magnitude, self.theta + other.theta)
    
    def __sub__(self, other):
        return PolarVector(self.magnitude - other.magnitude, self.theta - other.theta)
    
    def __str__(self):
        return "<" + str(self.magnitude) + ", " + str(self.theta) + ">"
    
def dot(a, b):
    return PolarVector(a.magnitude * b.magnitude, a.theta * b.theta)

class ComponentVector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __str__(self):
        return "<" + str(self.x) + ", " + str(self.y) + ">"
    
    def __mul__(self, other):
        return ComponentVector(self.x * other, self.y * other)
    
    def __add__(self, other):
        return ComponentVector(self.x + other.x, self.y + other.y)
    
    def __sub__(self, other):
        return ComponentVector(self.x - other.x, self.y - other.y)
    
    def __str__(self):
        return "<" + str(self.x) + ", " + str(self.y) + ">"
    
    

        
def polarToComponent(vector):
    y = math.sin(vector.theta)*vector.magnitude
    x = math.cos(vector.theta)*vector.magnitude
    
    return ComponentVector(x, y)

def componentToPolar(vector):
    magnitude = math.sqrt(vector.x**2 + vector.y**2)
    theta = math.atan(vector.y / vector.x)
    
    return PolarVector(magnitude, theta)


In [6]:
#Basic, helpful functions like unit conversions

def kmToAU(km):
    return km / 1.496e+8

def auToKm(au):
    return au * 1.496e+8

def auToM(au):
    return au * 1.496e+11



In [7]:
#Functions for solving gauss problem

import math

#the maximum test value for p, the conic parameter or semi-latus rectum
G_CONIC_PARAMETER_TEST_VALUE = 340649140000000000


class GaussReturnValue:
    def __init__(self, parameter, velocityInitial, velocityTarget):
        self.parameter = parameter
        self.velocityInitial = velocityInitial
        self.velocityTarget = velocityTarget
        

def findFDot(gravitationalParameter, semiMajorAxis, radiusInitial, radiusTarget, eccentricAnomaly):
    return (-(math.sqrt(gravitationalParameter * semiMajorAxis))/(radiusInitial * radiusTarget)) * math.sin(eccentricAnomaly)

def findGDot(semiMajorAxis, radiusTarget, eccentricAnomaly):
    return 1 - (semiMajorAxis / radiusTarget) * (1 - math.cos(eccentricAnomaly))


def findInitialVelocity(r1VectorPolar, r2VectorPolar, fFunction, gFunction):
    r1Vector = polarToComponent(r1VectorPolar)
    r2Vector = polarToComponent(r2VectorPolar)
    
    print("r1: " + str(r1Vector))
    print("r2: " + str(r2Vector))
    
    v1X = (r2Vector.x - fFunction * r1Vector.x) / gFunction
    v1Y = (r2Vector.y - fFunction * r1Vector.y) / gFunction
    
    print("velocity: " + str(v1X) + ", " + str(v1Y))
    
    v1Magnitude = math.sqrt(v1X**2 + v1Y**2)
    v1Theta = math.atan(v1Y/v1X)
    
    #return ComponentVector(v1X, v1Y)
    return PolarVector(v1Magnitude, v1Theta)


def findTargetVelocity(r1VectorPolar, v1VectorPolar, fDot, gDot):
    r1Vector = polarToComponent(r1VectorPolar)
    v1Vector = polarToComponent(v1VectorPolar)
    
    print("r1: " + str(r1Vector))
    print("v1: " + str(v1Vector))
    
    v2X = fDot * r1Vector.x + gDot * v1Vector.x
    v2Y = fDot * r1Vector.y + gDot * v1Vector.y
    
    v2Magnitude = math.sqrt(v2X**2 + v2Y**2)
    v2Theta = math.atan(v2Y/v2X)
    
    #return ComponentVector(v2X, v2Y)
    return PolarVector(v2Magnitude, v2Theta)

            

#solves the Gauss problem via the p-iteration technique
def solveGauss(gravitationalParameter, r1Vector, r2Vector, deltaTrueAnomaly, timeOfFlight):
    
    radiusInitial = r1Vector.magnitude
    radiusTarget = r2Vector.magnitude

    #generically named constants. I'm not sure exactly what these values represent
    kConstant = radiusInitial * radiusTarget * (1 - math.cos(deltaTrueAnomaly))
    lConstant = radiusInitial + radiusTarget
    mConstant = radiusInitial * radiusTarget * (1 + math.cos(deltaTrueAnomaly))
    
    
    #minimum and maximum values of the semi-latus rectum depending on value of the change in true anomaly
    parameterMin = 0
    parameterMax = 0
    
    if deltaTrueAnomaly < math.pi:
        parameterMin = kConstant / (lConstant + math.sqrt(2 * mConstant))
        parameterMax = G_CONIC_PARAMETER_TEST_VALUE
    else:
        parameterMax = kConstant / (lConstant - math.sqrt(2 * mConstant))
    
    while True:
        parameter = (parameterMin + parameterMax) / 2
        print("Testing for p value of " + str(parameter) + " (minimum of " + str(parameterMin) + ", maximum of " + str(parameterMax) + ")")
        
        semiMajorAxis = (mConstant * kConstant * parameter) / ((2 * mConstant - lConstant**2) * parameter**2 + 2 * kConstant * lConstant * parameter - kConstant**2)
        
        fFunction = 1 - (radiusTarget / parameter) * (1 - math.cos(deltaTrueAnomaly))
        gFunction = (radiusInitial * radiusTarget * math.sin(deltaTrueAnomaly)) / math.sqrt(gravitationalParameter * parameter)
        
        print("f: " + str(fFunction) + " g: " + str(gFunction))
        #test value for time-of-flight
        timeOfFlightTest = 0
                                                 
        eccentricAnomaly = 0 #leaving this here to put variable in wider scope
        
        if semiMajorAxis >= 0:
            print("Positive a")
            eccentricAnomaly = math.acos(1 - (radiusInitial / semiMajorAxis) * (1 - fFunction))
            
            timeOfFlightTest = gFunction + math.sqrt(semiMajorAxis**3 / gravitationalParameter) * (eccentricAnomaly - math.sin(eccentricAnomaly))
        else:
            eccentricAnomaly = math.acosh(1 - (radiusInitial / semiMajorAxis) * (1 - fFunction))
            
            timeOfFlightTest = gFunction + math.sqrt(((-semiMajorAxis)**3) / gravitationalParameter) * (math.sinh(eccentricAnomaly) - eccentricAnomaly)
        
        print("Testing t of " + str(timeOfFlightTest))
        if math.isclose(timeOfFlight, timeOfFlightTest, rel_tol=0.005):
            velocityInitial = findInitialVelocity(r1Vector, r2Vector, fFunction, gFunction)
            
            fDot = findFDot(gravitationalParameter, semiMajorAxis, radiusInitial, radiusTarget, eccentricAnomaly)
            print("ḟ: " + str(fDot))
            gDot = findGDot(semiMajorAxis, radiusTarget, eccentricAnomaly)
            print("ġ: " + str(gDot))
            velocityTarget = findTargetVelocity(r1Vector, velocityInitial, fDot, gDot)
                                                 
            return GaussReturnValue(parameter, velocityInitial, velocityTarget)
            
        elif timeOfFlightTest > timeOfFlight:
            parameterMin = parameter
        else:
            parameterMax = parameter
            
        
        


In [20]:
#Functions for determining orbital elements after solving gauss problem and delta v

class transferParametersReturnValue:
    def __init__(self, injectionDeltaV, zenith):
        self.injectionDeltaV = injectionDeltaV
        self.zenith = zenith


def getInjectionDeltaV(velocityInfinity, gravitationalParameter, parkingRadius, parkingVelocityMagnitude):
    injectionVelocityMagnitude = math.sqrt(velocityInfinity.magnitude**2 + ((2 * gravitationalParameter)/ parkingRadius))
    
    return injectionVelocityMagnitude - parkingVelocityMagnitude

def getZenithAngle(velocityInfinity, radiusInitial):
    velocityMagnitude = velocityInfinity.magnitude
    radiusMagnitude = radiusInitial.magnitude
    print("Mags: " + str(velocityMagnitude) + " " + str(radiusMagnitude))
    
    velocityInfinity = polarToComponent(velocityInfinity)
    radiusInitial = polarToComponent(radiusInitial)
    
    print(str(velocityInfinity) + " " + str(radiusInitial))
    
    return math.acos((radiusInitial.x * velocityInfinity.x + radiusInitial.y * velocityInfinity.y) / (radiusMagnitude * velocityMagnitude))


#outputs as component vector as conversion to polar can result in an incorrect output
def getApproachVector(approachDistance, planetPosition):
    planetPosition = polarToComponent(planetPosition)
    
    distanceX = (-approachDistance * planetPosition.y) / (math.sqrt(planetPosition.x**2 + planetPosition.y**2))
    distanceY = (approachDistance * planetPosition.x) / (math.sqrt(planetPosition.x**2 + planetPosition.y**2))
    
    return ComponentVector(distanceX, distanceY)

def getApproachAngle(approachVector, approachDistance, velocityInfinity):
    velocityMagnitude = velocityInfinity.magnitude

    velocityInfinity = polarToComponent(velocityInfinity)
    
    theta = math.acos((approachVector.x * velocityInfinity.x + approachVector.y + velocityInfinity.y) / (approachDistance * velocityMagnitude))
    
    return theta

def getImpactParameter(approachDistance, approachAngle):
    return approachDistance * math.sin(approachAngle)


def getApproachSemiMajorAxis(gravitationalParameter, velocityInfinity):
    return -(gravitationalParameter / velocityInfinity.magnitude**2)

def getApproachEccentricity(impactParameter, semiMajorAxis):
    return math.sqrt(1 + (impactParameter**2 / semiMajorAxis**2))


def getCaptureDeltaV():
    return


def getTransferParameters(velocityInitial, radiusInitial, planetVelocity, initialPlanetGravitationalParameter, parkingRadius, parkingVelocity):
    velocityInfinity = componentToPolar(polarToComponent(velocityInitial) - polarToComponent(planetVelocity))
    
    
    injectionDeltaV = getInjectionDeltaV(velocityInfinity, initialPlanetGravitationalParameter, parkingRadius, parkingVelocity)
    zenith = getZenithAngle(velocityInfinity, radiusInitial)
    
    return transferParametersReturnValue(injectionDeltaV, zenith,)
    

def getCaptureDeltaV(velocityTarget, radiusTarget, planetVelocity, planetPosition, targetPlanetGravitationalParameter, approachDistance, parkingEccentricity = 0):
    velocityInfinity = componentToPolar(polarToComponent(velocityTarget) - polarToComponent(planetVelocity))
    
    #print("v∞: " + str(polarToComponent(velocityInfinity)) + ", v2: " + str(polarToComponent(velocityTarget)) + ", vp: " + str(polarToComponent(planetVelocity)))
    
    approachVector = getApproachVector(approachDistance, planetPosition)
    approachAngle = getApproachAngle(approachVector, approachDistance, velocityInfinity)
    
    impactParameter = getImpactParameter(approachDistance, approachAngle)
    
    semiMajorAxis = getApproachSemiMajorAxis(targetPlanetGravitationalParameter, velocityInfinity)
    
    eccentricity = getApproachEccentricity(impactParameter, semiMajorAxis)
    
    print("d: " + str(approachVector) + ", θ: " + str(approachAngle) + ", b: " + str(impactParameter) + ", a: " + str(semiMajorAxis) + ", e: " + str(eccentricity))
    
    periapsisRadius = semiMajorAxis * (1 - eccentricity)
    
    print("r₀: " + str(periapsisRadius))
    
    periapsisVelocity = math.sqrt(targetPlanetGravitationalParameter * ((2 / periapsisRadius) - (1 / semiMajorAxis)))
    
    print("v at r₀: " + str(periapsisVelocity))
    
    desiredSemiMajorAxis = periapsisRadius / (1 - parkingEccentricity)
    
    desiredVelocity = math.sqrt(targetPlanetGravitationalParameter * ((2 / periapsisRadius) - (1 / desiredSemiMajorAxis)))
    
    return periapsisVelocity - desiredVelocity
    

In [21]:
#Value definitinos and running the program





#test values
gravitationalParameter =  3.964016e-14
radiusInitial =  1.016153
radiusTarget =  1.562993
deltaTrueAnomaly = 2.613996498039
timeOfFlight = 1.788e+7
phaseAngle = 0.9205

r1Vector = PolarVector(radiusInitial, -1.0863)
r2Vector = PolarVector(radiusTarget, 1.52800949)


#origin planet properties
originPlanetGravitationalParameter = 3.9834e14
originPlanetVelocity = componentToPolar(ComponentVector(25876.6, 13759.5))

#target planet properties
targetPlanetGravitationalParameter = 4.282831e13
targetPlanetVelocity = componentToPolar(ComponentVector(-23307, 3112))
targetPlanetPosition = PolarVector(auToM(r2Vector.magnitude), r2Vector.theta)

#intitial parking orbit properties
parkingOrbitRadius = 6578000
parkingOrbitVelocity = 7781.85

approachDistance = 18500000


gaussSolution = solveGauss(gravitationalParameter, r1Vector, r2Vector, deltaTrueAnomaly, timeOfFlight)

print("P-Value: " + str(gaussSolution.parameter) + ", v1: " + str(gaussSolution.velocityInitial) + ", v2: " + str(gaussSolution.velocityTarget))

velocityInitial = PolarVector(auToM(gaussSolution.velocityInitial.magnitude), gaussSolution.velocityInitial.theta)

r1VectorM = PolarVector(auToM(r1Vector.magnitude), r1Vector.theta)

transferParameters = getTransferParameters(velocityInitial, r1VectorM, originPlanetVelocity, originPlanetGravitationalParameter, parkingOrbitRadius, parkingOrbitVelocity)


velocityTarget = PolarVector(auToM(gaussSolution.velocityTarget.magnitude), gaussSolution.velocityTarget.theta)

r2VectorM = PolarVector(auToM(r2Vector.magnitude), r2Vector.theta)

captureDeltaV = getCaptureDeltaV(velocityTarget, r2Vector, targetPlanetVelocity, targetPlanetPosition, targetPlanetGravitationalParameter, approachDistance)



print("Delta-v: " + str(transferParameters.injectionDeltaV) + ", zenith angle: " + str(transferParameters.zenith) + ", capture delta-v: " + str(captureDeltaV))


Testing for p value of 1.7032457e+17 (minimum of 0.9147638652594379, maximum of 340649140000000000)
f: 1.0 g: 0.009731345346046178
Testing t of 0.009731345346046178
Testing for p value of 8.5162285e+16 (minimum of 0.9147638652594379, maximum of 1.7032457e+17)
f: 1.0 g: 0.013762200568514803
Testing t of 0.013762200568514803
Testing for p value of 4.25811425e+16 (minimum of 0.9147638652594379, maximum of 8.5162285e+16)
f: 0.9999999999999999 g: 0.019462690692092357
Testing t of 0.019462690692092357
Testing for p value of 2.129057125e+16 (minimum of 0.9147638652594379, maximum of 4.25811425e+16)
f: 0.9999999999999999 g: 0.027524401137029607
Testing t of 0.027524401137029607
Testing for p value of 1.0645285625e+16 (minimum of 0.9147638652594379, maximum of 2.129057125e+16)
f: 0.9999999999999998 g: 0.038925381384184714
Testing t of 0.038925381384184714
Testing for p value of 5322642812500000.0 (minimum of 0.9147638652594379, maximum of 1.0645285625e+16)
f: 0.9999999999999994 g: 0.05504880227

# # 1# 1# 